# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
# Add to for timing code
import timeit

#### Creating list of filepaths to process original event csv data files

In [8]:
# checking your current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # eliminate python notebook backup csv files generated
    if(root != "/home/workspace/event_data/.ipynb_checkpoints"):
    # join the file path and roots with the subdirectories using glob
        file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)
# Should be 30 files
len(file_path_list)

/home/workspace


30

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

##### Process csv using csv reader/writer

In [3]:
start = timeit.default_timer()
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_new csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
stop = timeit.default_timer()
print('Time: ', (stop-start))

Time:  0.5078429029999825


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


##### Process csv using pandas (same as the above cell using csv writer but using Pandas)

In [5]:
start = timeit.default_timer()
# Read in CSV with pandas instead for fun
col_names = ['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', \
       'level', 'location', 'sessionId', 'song', 'userId']
full_df = pd.DataFrame(columns = col_names)
for f in file_path_list:
    #print(f)
    df = pd.read_csv(f)
    df = df.dropna(subset=['artist']).reset_index(drop=True).drop(columns=['auth', 'method', 'page', 'registration', 'status', 'ts'])
    full_df = full_df.append(df)

# Convert to int32
full_df = full_df.astype({'userId':'int32', 'sessionId': 'int32'})
# round the length to 5 to copy the csv version
full_df = full_df.round({'length': 5})
# Check NaN
print(full_df.isna().sum().sum())
full_df.to_csv('event_datafile_new.csv', index = False)
stop = timeit.default_timer()
print('Time: ', (stop-start))

Time:  2.2619616659999906


In [6]:
# Pandas doesn't count column name which is 1 row diff
full_df.shape[0]

6820

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify_db 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify_db')
except Exception as e:
    print(e)

### Create tables to run the following queries. Since we are using Apache Cassandra, model the database tables on the queries to run.

## Create queries to ask the following three questions of the data

### 1. Give the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'



In [10]:
## Query 1:  Give the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# Description: Primary Key has two fields: sessionId is the partition key, and itemInSession is clustering key. Partitioning is done by sessionId and within that partition, rows are ordered by the itemInSession.
        
query = """
CREATE TABLE IF NOT EXISTS music_app_history 
(sessionId int, itemInSession int, artist_name varchar, song_title varchar, song_length float, PRIMARY KEY(sessionId, itemInSession))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)    

##### Below code uses csv reader but I used pandas instead...code works but didn't like that I had to cast items for query to work properly
##### Cells below this use pandas CSV reader method to load tables instead.

In [11]:
# Using CSV reader to read data and insert into DB
file = 'event_datafile_new.csv'

 col_names = ['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'sessionId', 'song', 'userId']
# Assign the INSERT statements into the `query` variable
query = "INSERT INTO music_app_history (sessionId, itemInSession, artist_name, song_title, song_length) "
query = query + "VALUES(%s, %s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [12]:
# Let's use my pandas file instead since this preserves dtypes
file = 'event_datafile_new.csv'
query = "INSERT INTO music_app_history (sessionId, itemInSession, artist_name, song_title, song_length) "
query = query + "VALUES(%s, %s, %s, %s, %s)"
data_df = pd.read_csv(file)
for row in data_df.itertuples(index=False):
    #print(row[1])
    session.execute(query, (row[8], row[3], row[0], row[9], row[5]))

#### Query to find the artist and song title and song length for session Id: 338 and for the fourth song played during the session.

In [12]:
## Add in the SELECT statement to verify the data was entered into the table
query_verify = "SELECT artist_name, song_title, song_length FROM music_app_history WHERE sessionId=338 AND itemInSession=4"
try:
    rows = session.execute(query_verify)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Create table and query to answer question 2.

In [13]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# Description: Primary Key has three fields: userId is the partition key, and sessionId and itemInSession are the clustering keys. Partitioning is done by userId and within that partition, rows are ordered by the sessionId and then by itemInSession.
query2 = """
CREATE TABLE IF NOT EXISTS user_app_history 
(userId int, sessionId int, itemInSession int, artist_name varchar, song_title varchar, first_name varchar, last_name varchar, PRIMARY KEY(userId, sessionId, itemInSession))
"""
try:
    session.execute(query2)
except Exception as e:
    print(e)         

In [14]:
# col_names = ['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'sessionId', 'song', 'userId']
file = 'event_datafile_new.csv'
query = "INSERT INTO user_app_history (userId, sessionId, itemInSession, artist_name, song_title, first_name, last_name) "
query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
data_df = pd.read_csv(file)
for row in data_df.itertuples(index=False):
    #print(row[1])
    session.execute(query, (row[10], row[8], row[3], row[0], row[9], row[1], row[4]))
    
#print(data_df.head(1))

#### Query to find the artists and song titles and user first and last name for user Id: 10 and session Id 182.
##### Query will list all of the songs played in that session by order played using itemInSession

In [16]:
query_verify = "SELECT artist_name, song_title, first_name, last_name FROM user_app_history WHERE userId=10 AND sessionId=182"
try:
    rows = session.execute(query_verify)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [17]:
## Query 3: Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Description: Primary Key has two fields: song_title is the partition key, and userId is the clustering key. Partitioning is done by song_title and within that partition, rows are ordered by the userId.

query3 = """
CREATE TABLE IF NOT EXISTS song_app_history 
(song_title varchar, userId int, artist_name varchar, first_name varchar, last_name varchar, PRIMARY KEY(song_title, userId))
"""
try:
    session.execute(query3)
except Exception as e:
    print(e)       

In [18]:
# col_names = ['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'sessionId', 'song', 'userId']
file = 'event_datafile_new.csv'
query = "INSERT INTO song_app_history (song_title, userId, first_name, last_name) "
query = query + "VALUES(%s, %s, %s, %s)"
data_df = pd.read_csv(file)
for row in data_df.itertuples(index=False):
    #print(row[1])
    session.execute(query, (row[9], row[10], row[1], row[4]))
    
#print(data_df.head(1))

#### Query to find the names of users (first and last) who listened to the song title "All Hands Against His Own"

In [19]:
query_verify = "SELECT first_name, last_name FROM song_app_history WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query_verify)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [4]:
## Drop the table before closing out the sessions

In [20]:
query_drop = "DROP TABLE IF EXISTS music_app_history"
try:
    rows = session.execute(query_drop)
except Exception as e:
    print(e)

query_drop = "DROP TABLE IF EXISTS user_app_history"
try:
    rows = session.execute(query_drop)
except Exception as e:
    print(e)
    
query_drop = "DROP TABLE IF EXISTS song_app_history"
try:
    rows = session.execute(query_drop)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()